junto aca todos los ETL: 



In [1]:
import pandas as pd
#import numpy as np
import pandas_profiling


PUEDE FALTAR NORMALIZACION DE STRINGS TIPO: Sao Paulo, sao paulo... VER!

In [2]:
#aca hay q ver como los consume el PIPELINE: 

#carga de archivos como dfs.
df_olist_closed_list = pd.read_csv('Datasets/olist_closed_deals_dataset.csv')
df_customers_datasets = pd.read_csv('Datasets/olist_customers_dataset.csv')
df_olist_geolocation_dataset = pd.read_csv('Datasets/olist_geolocation_dataset.csv')
df_olist_marketing_qualified_leads_dataset = pd.read_csv('Datasets/olist_marketing_qualified_leads_dataset.csv')
df_olist_order_items_dataset = pd.read_csv('Datasets/olist_order_items_dataset.csv')
df_olist_order_payments_dataset = pd.read_csv('Datasets/olist_order_payments_dataset.csv')
df_olist_order_reviews_dataset = pd.read_csv('Datasets/olist_order_reviews_dataset.csv')
df_olist_orders_dataset = pd.read_csv('Datasets/olist_orders_dataset.csv')
df_olist_products_dataset = pd.read_csv('Datasets/olist_products_dataset.csv')
df_olist_sellers_dataset = pd.read_csv('Datasets/olist_sellers_dataset.csv')
df_product_category_name_translation = pd.read_csv('Datasets/product_category_name_translation.csv')

In [3]:
#df1 'df_olist_closed_list':
df_olist_closed_list=df_olist_closed_list.drop_duplicates()

#SON SIGNIFICADOS UNIVERSALES! ver si llegamos a utilizarlo en el analisis!
df_olist_closed_list = df_olist_closed_list.drop(['lead_behaviour_profile'], axis = 1)

#columnas con gran % de nulos: 
df_olist_closed_list=df_olist_closed_list.drop(['declared_product_catalog_size', 'has_company', 'has_gtin', 'average_stock'], axis = 1)

#Cuenta con un 94.7% de ceros
df_olist_closed_list=df_olist_closed_list.drop(['declared_monthly_revenue'], axis = 1)

#Cambio el tipo de dato a datetime
df_olist_closed_list['won_date'] = pd.to_datetime(df_olist_closed_list['won_date'])

df_olist_closed_list['business_segment'] = df_olist_closed_list['business_segment'].astype('string')
#Cambio el tipo a string

df_olist_closed_list['lead_type'] = df_olist_closed_list['lead_type'].astype('string')

df_olist_closed_list['business_type'] = df_olist_closed_list['business_type'].astype('string')


In [4]:
#df2 'df_customers_datasets':
df_customers_datasets.drop_duplicates()

#VER! si hace falta p algun kpi. Desconocemos realmente significado. 
df_customers_datasets = df_customers_datasets.drop('customer_unique_id', axis = 1)

df_customers_datasets['customer_zip_code_prefix'] = df_customers_datasets['customer_zip_code_prefix'].astype('int64')

df_customers_datasets['customer_city'] = df_customers_datasets['customer_city'].astype('string')

df_customers_datasets['customer_state'] = df_customers_datasets['customer_state'].astype('string')

In [5]:
#df3 'df_olist_geolocation_dataset':
df_olist_geolocation_dataset.drop_duplicates()

#ACA FALTA ACOTAR ! VER MAX Y MIN DE LAT LONG BRASIL!

#Cambio el tipo de dato a string
df_olist_geolocation_dataset['geolocation_city'] = df_olist_geolocation_dataset['geolocation_city'].astype('string')
df_olist_geolocation_dataset['geolocation_state'] = df_olist_geolocation_dataset['geolocation_state'].astype('string')

# Al visualizar rapidamente con PBI, notamos que muchas geolocalizciones estan fuera del territorio brasilero. Las descartamos al 
#acotarlas con datos de la web sobre limites: (ojo puede 'quedar alguna!', sino ver bookmark API zipcodes brazil)

#most Northern spot is at 5 deg 16′ 27.8″ N lat:
df_olist_geolocation_dataset = df_olist_geolocation_dataset[df_olist_geolocation_dataset.geolocation_lat <= 5.27438888]
#most Western spot is at 73 deg, 58′ 58.19″W Long:
df_olist_geolocation_dataset = df_olist_geolocation_dataset[df_olist_geolocation_dataset.geolocation_lng >= -73.98283055]
#most southern spot is at 33 deg, 45′ 04.21″ S Lat:
df_olist_geolocation_dataset = df_olist_geolocation_dataset[df_olist_geolocation_dataset.geolocation_lat >= -33.75116944]
#most Eastern spot is 34 deg, 47′ 35.33″ W Long:
df_olist_geolocation_dataset = df_olist_geolocation_dataset[df_olist_geolocation_dataset.geolocation_lng <=  -34.79314722]



In [6]:
#df4 'df_olist_marketing_qualified_leads_dataset':
df_olist_marketing_qualified_leads_dataset.drop_duplicates()

#Cambio el tipo de dato a datetime
df_olist_marketing_qualified_leads_dataset['first_contact_date'] = pd.to_datetime(df_olist_marketing_qualified_leads_dataset['first_contact_date'])

#Cambio el tipo de dato a string
df_olist_marketing_qualified_leads_dataset['origin'] = df_olist_marketing_qualified_leads_dataset['origin'].astype('string')

#df_olist_marketing_qualified_leads_dataset["origin"].fillna("No info", inplace = True)


In [7]:
#df5 'df_olist_order_items_dataset':

df_olist_order_items_dataset.drop_duplicates()
#cambio tipos de dato: 
df_olist_order_items_dataset['price'] = df_olist_order_items_dataset['price'].astype('float')
df_olist_order_items_dataset['freight_value'] = df_olist_order_items_dataset['freight_value'].astype('float')
df_olist_order_items_dataset['shipping_limit_date'] = pd.to_datetime(df_olist_order_items_dataset['shipping_limit_date'])


In [8]:
#df6 'df_olist_order_payments_dataset':

#Origen de ip_order (cruzado con df_olist_order_items_dataset, df_olist_order_reviews_dataset, df_olist_orders_dataset)
df_olist_order_payments_dataset.drop_duplicates()
#cambio tipo dato:
df_olist_order_payments_dataset['payment_type'] = df_olist_order_payments_dataset['payment_type'].astype('string')

In [9]:
#df7: 'df_olist_order_reviews_dataset'.

#borro duplicados si los hubiera: 
df_olist_order_reviews_dataset.drop_duplicates()

#columnas que no necesitamos: 

#'review_comment_title': 88.3% missing values! Por mas de presentar mayoria de falatantes. por ahora la dejo. 

#'review_comment_message': 58.7% missing!. igualmente, sirve para analisis sentimiento!

#'review_answer_timestamp' no aporta informacion relevante: CONSERVO: 'review_creation_date'. IGUAL PUEDE NO SER RELEVANTE. 
df_olist_order_reviews_dataset.drop(['review_answer_timestamp'], axis = 1, inplace=True)


#CHECK Y CAMBIO DE TIPOS DE DATO: 
# 'review_score': ya es int!

#df_olist_order_reviews_dataset=df_olist_order_reviews_dataset.astype({'review_score':int})

# tiro los duplicados de 'review_id', y 'order_id'. Del EDA se concluye corresponden a mismo review, enviado dos veces o simil. 
# no represntan gran porcentaje del total (99.2% unique y 99.4% unique respectivamente. )
df_olist_order_reviews_dataset.drop_duplicates(subset='review_id', keep="first", inplace=True)
df_olist_order_reviews_dataset.drop_duplicates(subset='order_id', keep="first", inplace=True)


#PARA MODELO ML: VER: Quedarme solo con los no nulos en 'review_comment_message' y 'review_comment_title'.
#from deep_translator import GoogleTranslator
# NO ACEPTA NULOS! PARA EL ML, hacer nueva tabla sin los nulos. y luego traducir!
#traductor = GoogleTranslator(source='pt', target='es')
#df_olist_order_reviews_dataset['review_title_espaniol']=df_olist_order_reviews_dataset.review_comment_title.apply(lambda x: traductor.translate(x))



In [10]:
#df8: 'df_olist_orders_dataset'.

#Borramos duplicados,
# no borramos columnas porque todas nos van a ser de utilidad para el análisis.
df_olist_orders_dataset = df_olist_orders_dataset.drop_duplicates()

#VER SI HACE FALTA: 
#df_olist_orders_dataset.fillna('Sin Dato', inplace=True)

#Convertimos el tipo de dato a datetime de las columnas de fechas que estaban como 'object':
#ORDENAR CODIGO PARA Q SEA EN UNA LINEA CON UNA LISTA!
df_olist_orders_dataset['order_purchase_timestamp'] = pd.to_datetime(df_olist_orders_dataset['order_purchase_timestamp'])
df_olist_orders_dataset['order_approved_at'] = pd.to_datetime(df_olist_orders_dataset['order_approved_at'],errors='coerce')
df_olist_orders_dataset['order_delivered_carrier_date'] = pd.to_datetime(df_olist_orders_dataset['order_delivered_carrier_date'],errors='coerce')
df_olist_orders_dataset['order_delivered_customer_date'] = pd.to_datetime(df_olist_orders_dataset['order_delivered_customer_date'],errors='coerce')
df_olist_orders_dataset['order_estimated_delivery_date'] = pd.to_datetime(df_olist_orders_dataset['order_estimated_delivery_date'])
df_olist_orders_dataset['order_status'] = df_olist_orders_dataset['order_status'].astype(str)

#PARA VIZ, DELIVERY TIME: adaptar!
#orders['actual_delivery_time'] = orders.order_delivered_customer_date - orders.order_delivered_carrier_date
#rders['actual_delivery_time'] = orders['actual_delivery_time'].dt.days



In [11]:
#df9 'df_olist_products_dataset':

df_olist_products_dataset = df_olist_products_dataset.drop_duplicates()

df_olist_products_dataset=df_olist_products_dataset.drop(['product_name_lenght','product_description_lenght','product_photos_qty'], axis = 1)


In [12]:
# Hay dos categorias de productos que no tienen su traduccion (unique diferia en 2, 71 y 73 valores al hacer el EDA)
#identifico: 
df_olist_products_dataset[~df_olist_products_dataset['product_category_name'].isin(df_product_category_name_translation['product_category_name'].unique())]['product_category_name'].value_counts()

#Output: 
#portateis_cozinha_e_preparadores_de_alimentos    10
#pc_gamer                                          3
#Name: product_category_name, dtype: int64

#portateis_cozinha_e_preparadores_de_alimentos english translation: is kitchen_laptops_and_food_preparators
#pc_gamer: pc_gamer.

#los agrego en la celda de df11, 'traducciones'.



portateis_cozinha_e_preparadores_de_alimentos    10
pc_gamer                                          3
Name: product_category_name, dtype: int64

In [ ]:
#df10 'df_olist_sellers_dataset':
df_olist_sellers_dataset=df_olist_sellers_dataset.drop_duplicates()

#FALTA CHECKEAR NORMALIZAR POR MINUSCULAS/MAYUSCULAS??

In [13]:
#df11 'df_product_category_name_translation':

#primero le agrego las dos que faltaban: 
traducciones = pd.DataFrame(data = {'product_category_name': ['portateis_cozinha_e_preparadores_de_alimentos', 'pc_gamer'],
                                    'product_category_name_english': ['kitchen_laptops_and_food_preparators', 'pc_gamer']},
                                    index = [71, 72])
df_product_category_name_translation = df_product_category_name_translation.append(traducciones)
#df_product_category_name_translation.tail(2)
#OUTPUT: 
#product_category_name	product_category_name_english
#71	portateis_cozinha_e_preparadores_de_alimentos	kitchen_laptops_and_food_preparators
#72	pc_gamer	pc_gamer

#hago un MERGE con tabla de productos para que ya quede todo en ingles, descarto la col en portugues, 
# y la renombro: OJO VER CARGAS INCREMENTALES FALLA???!
df_olist_products_dataset = pd.merge(df_olist_products_dataset,df_product_category_name_translation,on="product_category_name")
df_olist_products_dataset.drop("product_category_name",inplace=True,axis=1)
df_olist_products_dataset.rename(columns={"product_category_name_english":"product_category_name"},inplace=True)

/var/folders/tf/dsk90f9d14gcg40_frclnb7r0000gn/T/ipykernel_35754/1832352030.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_product_category_name_translation = df_product_category_name_translation.append(adding_trans)


In [15]:
#df_olist_products_dataset.head()